# Quick Start

In this notebook, we will help you be familiar with Evox through an simple example.

To start with, make sure you have already installed the EvoX.

In [1]:
# install evox, skip it if you have already installed evox
try:
    import evox
except ImportError:
    !pip install --disable-pip-version-check --upgrade -q evox

In [4]:
# Set an monitor
monitor = EvalMonitor()

## Initiate an workflow

A **workflow** outlines the series of steps required to accomplish a task or project. In EvoX, a workflow represents the overall process of evolutionary computation, putting the algorithm, problem, monitor together.

If we want to run the algorithm to solve the problem with a monitor, we need to create a workflow of the `Workflow` class.

In [5]:
# Initiate an workflow
workflow = StdWorkflow()
workflow.setup(
    algorithm=algorithm,
    problem=problem,
    monitor=monitor,
    )

## Run the workflow

Now, we can run the workflow:

In [6]:
start = time.time()

# Run the workflow
for i in range(1,101):
    workflow.step()
    if i % 10 == 0:
        run_time = time.time() - start
        top_fitness = workflow.get_submodule("monitor").topk_fitness
        print(f"The top fitness is {top_fitness} in {run_time:.4f} seconds at the {i}th generation.")

The top fitness is tensor([0.0631]) in 0.1246 seconds at the 10th generation.
The top fitness is tensor([0.0158]) in 0.1291 seconds at the 20th generation.
The top fitness is tensor([0.0026]) in 0.1311 seconds at the 30th generation.
The top fitness is tensor([0.0026]) in 0.1341 seconds at the 40th generation.
The top fitness is tensor([0.0026]) in 0.1361 seconds at the 50th generation.
The top fitness is tensor([0.0026]) in 0.1390 seconds at the 60th generation.
The top fitness is tensor([0.0026]) in 0.1421 seconds at the 70th generation.
The top fitness is tensor([0.0026]) in 0.1441 seconds at the 80th generation.
The top fitness is tensor([0.0026]) in 0.1471 seconds at the 90th generation.
The top fitness is tensor([0.0012]) in 0.1491 seconds at the 100th generation.


Congretulations if you can reproduce the Quick Start demo! Hope you enjoy the EvoX!